In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rtatman/english-word-frequency")

print("Path to dataset files:", path)

Path to dataset files: /home/gian/.cache/kagglehub/datasets/rtatman/english-word-frequency/versions/1


In [28]:
from polars import read_csv

words_dataframe = read_csv(path + "/unigram_freq.csv")
# list of words
words = [ w for w in words_dataframe["word"][0:50000] if len(w) > 1]

bigrams = {}
for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for b in zip(chs, chs[1:]):
        bigrams[b] = bigrams.get(b, 0) + 1

In [29]:
bigrams = sorted(bigrams.items(), key=lambda x: x[1], reverse=True)
bigrams

[(('s', '<E>'), 9627),
 (('i', 'n'), 6664),
 (('e', 'r'), 6412),
 (('e', '<E>'), 6134),
 (('<S>', 's'), 5018),
 (('o', 'n'), 4717),
 (('<S>', 'c'), 4645),
 (('r', 'e'), 4490),
 (('e', 's'), 4489),
 (('t', 'i'), 4241),
 (('d', '<E>'), 4162),
 (('t', 'e'), 4150),
 (('n', '<E>'), 4137),
 (('a', 'n'), 4121),
 (('e', 'n'), 3889),
 (('a', 't'), 3727),
 (('a', 'r'), 3588),
 (('<S>', 'p'), 3540),
 (('n', 'g'), 3466),
 (('e', 'd'), 3406),
 (('s', 't'), 3364),
 (('l', 'e'), 3342),
 (('<S>', 'a'), 3331),
 (('y', '<E>'), 3270),
 (('r', 'a'), 3258),
 (('a', 'l'), 3215),
 (('r', '<E>'), 3177),
 (('o', 'r'), 3081),
 (('<S>', 'm'), 3075),
 (('t', '<E>'), 2999),
 (('g', '<E>'), 2982),
 (('r', 'i'), 2978),
 (('<S>', 'b'), 2874),
 (('n', 't'), 2800),
 (('<S>', 'd'), 2758),
 (('l', 'i'), 2683),
 (('<S>', 'r'), 2658),
 (('c', 'o'), 2569),
 (('<S>', 't'), 2545),
 (('d', 'e'), 2505),
 (('a', '<E>'), 2458),
 (('n', 'e'), 2447),
 (('i', 's'), 2421),
 (('r', 'o'), 2373),
 (('i', 'c'), 2355),
 (('l', 'a'), 2277)